# M1 - A First Example
We want to get the daily closing stock prices from 2016 to 2020 for the major players in the distilled and blended liquor industry. These are Brown-Forman Corporation (BF-B), Diageo (DEO), Eastside Distilling (EAST), and MGP Ingredients (MGPI). After retrieving the data, we want to make a simple line plot for each company.

In [ ]:
# We are going to use a package called 'yahooquery'
# It pulls the data from Yahoo Finance.
from yahooquery import Ticker

%matplotlib inline

In [ ]:
# We create a Ticker object by passing it a list of four ticker symbols
# The second parameter passed in tells the code to make requests to
# Yahoo Finance asynchronously (need to do this for when querying
# multiple ticker symbols).

tickers = Ticker("BF-B DEO EAST MGPI", asynchronous=True)

In [ ]:
# Get the daily stock price history starting January 1, 2016
# and ending December 31, 2020. The result will be a pandas
# DataFrame, which I am calling 'df'. We can look at the first
# few rows using the .head() function.

df = tickers.history(start="2016-01-01", end="2020-12-31")
df.head()

In [ ]:
# Take a peek at what the dataframe looks like
# We see can see all of the columns of the dataframe, along with how
# how many rows there are as well as the datatype for each column.
# Useful information!

df.info()

In [ ]:
# We can take a look at the index of the dataframe.
# We have a MultiIndex -- this is also considered a hierarchal index.
# Unfortunately, for our simple desire of a line plot, this type of index
# makes it slightly more "involved" to get our graph

df.index

In [ ]:
# We can also list all of the columns of our dataframe
df.columns

In [ ]:
# Let's look at Brown-Forman (because they make Woodford Reserve).
# We can use the .loc() function to pull out just BF-B.
df.loc[("BF-B")]

In [ ]:
# We can easily plot BF-B closing price over the 5-year period.
# The pandas dataframe let's us plot directly with it ... pretty cool!
df.loc[("BF-B")].plot(y="close")

In [ ]:
# What happens if we try to plot the closing price for everything?
# This plots only a single line ... 
df['close'].plot()

## No Good
That result does not look correct. I wanted to see 4 different lines, one for each ticker symbol. Instead, I got a single line. Here is where the <code>MultiIndex</code> on our original dataframe is causing us some pain. In short, we want each ticker symbol to be its own column in order for the plotting function to create separate lines.

### One Approach to Fix
Let's create a new <code>DataFrame</code> that stores the just the closing stock price over the 5-year period, but make each column one of the ticker symbols.

#### Step 1.
Pull out the closing stock price for BF-B as a <code>Series</code>, rename it, and make it a <code>DataFrame</code> called <code>myDF</code>.

#### Step 2.
Add DEO, EAST, and MGPI to <code>myDF</code> each as its own column, naming them correctly.

#### Step 3.
Look at our <code>myDF</code> to see what it looks like

In [ ]:
# Step 1.
myDF = df.loc["BF-B"]["close"].rename("BF-B").to_frame()

# Step 2.
myDF["DEO"] = df.loc["DEO"]["close"]
myDF["EAST"] = df.loc["EAST"]["close"]
myDF["MGPI"] = df.loc["MGPI"]["close"]

# Step 3.
myDF

## Time to Plot
Let's use the package <code>seaborn</code> to plot the four time series. I'll set the style to <code>ticks</code> to clean up the plot a little. Additionally, we'll make the color palette colorblind-friendly. Finally, I <code>despine</code> the plot which is simply removing the top and right lines of the graph region.

In [ ]:
import seaborn as sns
sns.set_style("ticks")

sns.lineplot(data=myDF, palette="colorblind")
sns.despine()

### Another Approach to Fix
Let's create a new `DataFrame` that stores the just the closing stock price over the 5-year period, but make each column one of the ticker symbols.

#### Step 1.
Pull out the closing stock price for all of the tickers as `Series`, rename it, and make it a <code>DataFrame</code> called `justClose`.

#### Step 2.
Reset the index of `justClose` so that we remove the `MultiIndex`.

#### Step 3.
Pivot the data in `justClose` so that we have each symbol as its own column.

In [ ]:
# Step 1. Pull out just the closing stock price for every stock of every day
justClose = df["close"].to_frame()
justClose

In [ ]:
# Step 2. Reset the index removing the multi-index
justClose.reset_index(inplace=True)
justClose

In [ ]:
# Step 3. Pivot the data on "date", making each symbol its own column, 
# using the closing price as the value
pivotedJC = justClose.pivot(index="date", columns="symbol", values="close")
pivotedJC

In [ ]:
# Looks good. So, let's plot using native dataframe capability
pivotedJC.plot()

In [ ]:
# We can also plot with seaborn like we did before.
sns.set_style("ticks")

sns.lineplot(data=pivotedJC, palette="colorblind")
sns.despine()

**&copy; 2021 - Present: Matthew D. Dean, Ph.D.   
Clinical Associate Professor of Business Analytics at William \& Mary.**